Calling weather data for chosen month/year and locations

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import *  # package for unzipping zip files
from sql_functions import get_engine


In [3]:
# To be able to pull the API key from the .env
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import json
import pandas as pd


In [5]:
# Set the url
#url = 'http://api.openweathermap.org/data/2.5/weather?'
url = "https://meteostat.p.rapidapi.com/stations/hourly"
#url = "https://meteostat.p.rapidapi.com/stations/daily"

# Set locations
locations = ["72205", 
             "72219", 
             "72221", 
             "72243",
             "72530"]

headers = {
			"X-RapidAPI-Key" : os.getenv('x_rapid_api_key'),
	        "X-RapidAPI-Host": "meteostat.p.rapidapi.com"
}

import time

# Create empty dataframe, will be used to append each location's weather data
weather_df = pd.DataFrame([])

for location in locations:
    querystring = { "station": location,
                "start": "2014-01-25",
                "end": "2014-01-31"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    #weather_df[location] = response
    time.sleep(15)
    # Decode repsonse with json decoder
    weather_temp = response.json()

    # Flatten json response
    weather_temp_df = pd.json_normalize(weather_temp, 
                                        sep="_", 
                                        record_path="data",
                                         meta=[["meta", "generated"],
                                         ], 
                                         record_prefix="data_", 
                                         errors='ignore')
    # concatenate dataframes
    weather_temp_df['station_id'] = location
    weather_df = pd.concat([weather_df, weather_temp_df], ignore_index=True)

# Print final dataset weather_df
weather_df

,data_time,data_temp,data_dwpt,data_rhum,data_prcp,data_snow,data_wdir,data_wspd,data_wpgt,data_pres,data_tsun,data_coco,meta_generated,station_id
0,2014-01-25 00:00:00,8.9,1.1,58.0,0.0,None,10.0,20.5,None,1028.0,None,None,2022-12-05 16:29:21,72205
1,2014-01-25 01:00:00,8.3,1.0,60.0,0.0,None,10.0,22.3,None,1027.8,None,None,2022-12-05 16:29:21,72205
2,2014-01-25 02:00:00,7.2,1.1,65.0,0.0,None,350.0,20.5,None,1027.6,None,None,2022-12-05 16:29:21,72205
3,2014-01-25 03:00:00,7.2,1.1,65.0,0.0,None,340.0,9.4,None,1027.5,None,None,2022-12-05 16:29:21,72205
4,2014-01-25 04:00:00,7.2,1.1,65.0,0.0,None,340.0,11.2,None,1027.0,None,None,2022-12-05 16:29:21,72205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,2014-01-31 19:00:00,-5.0,-12.9,54.0,0.0,None,230.0,9.4,None,1020.5,None,None,2022-12-05 15:26:57,72530
836,2014-01-31 20:00:00,-5.0,-13.3,52.0,0.0,None,260.0,13.0,None,1020.5,None,None,2022-12-05 15:26:57,72530
837,2014-01-31 21:00:00,-5.0,-12.9,54.0,0.0,None,250.0,11.2,None,1020.0,None,None,2022-12-05 15:26:57,72530
838,2014-01-31 22:00:00,-5.0,-12.2,57.0,0.0,None,270.0,9.4,None,1020.5,None,None,2022-12-05 15:26:57,72530


In [6]:
weather_df

,data_time,data_temp,data_dwpt,data_rhum,data_prcp,data_snow,data_wdir,data_wspd,data_wpgt,data_pres,data_tsun,data_coco,meta_generated,station_id
0,2014-01-25 00:00:00,8.9,1.1,58.0,0.0,None,10.0,20.5,None,1028.0,None,None,2022-12-05 16:29:21,72205
1,2014-01-25 01:00:00,8.3,1.0,60.0,0.0,None,10.0,22.3,None,1027.8,None,None,2022-12-05 16:29:21,72205
2,2014-01-25 02:00:00,7.2,1.1,65.0,0.0,None,350.0,20.5,None,1027.6,None,None,2022-12-05 16:29:21,72205
3,2014-01-25 03:00:00,7.2,1.1,65.0,0.0,None,340.0,9.4,None,1027.5,None,None,2022-12-05 16:29:21,72205
4,2014-01-25 04:00:00,7.2,1.1,65.0,0.0,None,340.0,11.2,None,1027.0,None,None,2022-12-05 16:29:21,72205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,2014-01-31 19:00:00,-5.0,-12.9,54.0,0.0,None,230.0,9.4,None,1020.5,None,None,2022-12-05 15:26:57,72530
836,2014-01-31 20:00:00,-5.0,-13.3,52.0,0.0,None,260.0,13.0,None,1020.5,None,None,2022-12-05 15:26:57,72530
837,2014-01-31 21:00:00,-5.0,-12.9,54.0,0.0,None,250.0,11.2,None,1020.0,None,None,2022-12-05 15:26:57,72530
838,2014-01-31 22:00:00,-5.0,-12.2,57.0,0.0,None,270.0,9.4,None,1020.5,None,None,2022-12-05 15:26:57,72530


In [7]:
import pandas as pd
import numpy as np
import psycopg2 # needed to get database exception errors when uploading dataframe
import requests # package for getting data from the web
from zipfile import *  # package for unzipping zip files
from sql_functions import get_engine

schema = 'cgn_analytics_22_3' # UPDATE 'TABLE_SCHEMA' based on schema used in class 
engine = get_engine() # assign engine to be able to query against the database

# Specify which table within your database you want to push your data to. Give your table an unambiguous name.
table_name = 'gp3_weatherdata_hourly'

# If the specified table doesn't exist yet, it will be created
# With 'replace', your data will be replaced if the table already exists.
# This may take some time ...

# Write records stored in a dataframe to SQL database
if engine!=None:
    try:
        weather_df.to_sql(name=table_name, # Name of SQL table
                        con=engine, # Engine or connection
                        if_exists='replace', # Drop the table before inserting new values 
                        schema=schema, # Use schmea that was defined earlier
                        index=False, # Write DataFrame index as a column
                        chunksize=5000, # Specify the number of rows in each batch to be written at a time
                        method='multi') # Pass multiple values in a single INSERT clause
        print(f"The {table_name} table was imported successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None


The weather_data_gp3ns table was imported successfully.
